In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
# import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from sklearn.metrics import accuracy_score, classification_report


In [2]:
df = pd.read_csv("./data/modified/previous_flight_status_data.csv")
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,timestamp_local_x,snow_rate_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,timestamp_local_y,snow_rate_y,key_y,date_time_map_y,arrival_status,previous_flight_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,2020-01-01 08:15:00,0.00,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,2020-01-01 07:00:00,0.00,JFK,2020-01-01 07:00:00,early,NaN
1,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,2020-01-01 11:30:00,0.00,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,2020-01-01 10:00:00,0.00,JFK,2020-01-01 10:00:00,early,early
2,YX,01/01/2020,5957,JFK,18:02,17:35,93,68,-27,4,0,0,0,0,0,2020-01-01 18:02:00,2020-01-01 17:35:00,2020-01-01,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,2020-01-01 18:00:00,0.00,SYR,2020-01-01 18:00:00,SYR,16:29,16:27,93,68,-2,13,0,0,0,0,0,2020-01-01 16:29:00,2020-01-01 16:27:00,2020-01-01,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,2020-01-01 16:30:00,0.00,JFK,2020-01-01 16:30:00,early,early
3,B6,01/01/2020,2516,JFK,20:50,20:31,80,71,-19,4,0,0,0,0,0,2020-01-01 20:50:00,2020-01-01 20:31:00,2020-01-01,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,2020-01-01 20:45:00,0.00,SYR,2020-01-01 20:45:00,SYR,19:30,19:20,80,71,-10,19,0,0,0,0,0,2020-01-01 19:30:00,2020-01-01 19:20:00,2020-01-01,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,2020-01-01 19:30:00,0.00,JFK,2020-01-01 19:30:00,early,early
4,B6,01/02/2020,116,JFK,08:22,08:06,75,67,-16,3,0,0,0,0,0,2020-01-02 08:22:00,2020-01-02 08:06:00,2020-01-02,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,2020-01-02 08:15:00,0.00,SYR,2020-01-02 08:15:00,SYR,07:07,06:59,75,67,-8,24,0,0,0,0,0,2020-01-02 07:07:00,2020-01-02 06:59:00,2020-01-02,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,2020-01-02 07:00:00,0.00,JFK,2020-01-02 07:00:00,early,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,UA,12/30/2023,783,ORD,23:59,00:13,114,104,14,5,0,0,0,0,0,2023-12-30 23:59:00,2023-12-31 00:13:00,2023-12-30,2023-12-30 23:59:00,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,2023-12-31 00:00:00,0.00,SYR,2023-12-31 00:00:00,SYR,21:05,21:29,114,104,24,17,0,0,0,0,0,2023-12-30 22:05:00,2023-12-30 22:29:00,2023-12-30,2023-12-30

In [3]:
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,timestamp_local_x,snow_rate_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,timestamp_local_y,snow_rate_y,key_y,date_time_map_y,arrival_status,previous_flight_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,2020-01-01 08:15:00,0.00,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,2020-01-01 07:00:00,0.00,JFK,2020-01-01 07:00:00,early,NaN
1,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,2020-01-01 11:30:00,0.00,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,2020-01-01 10:00:00,0.00,JFK,2020-01-01 10:00:00,early,early
2,YX,01/01/2020,5957,JFK,18:02,17:35,93,68,-27,4,0,0,0,0,0,2020-01-01 18:02:00,2020-01-01 17:35:00,2020-01-01,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,2020-01-01 18:00:00,0.00,SYR,2020-01-01 18:00:00,SYR,16:29,16:27,93,68,-2,13,0,0,0,0,0,2020-01-01 16:29:00,2020-01-01 16:27:00,2020-01-01,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,2020-01-01 16:30:00,0.00,JFK,2020-01-01 16:30:00,early,early
3,B6,01/01/2020,2516,JFK,20:50,20:31,80,71,-19,4,0,0,0,0,0,2020-01-01 20:50:00,2020-01-01 20:31:00,2020-01-01,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,2020-01-01 20:45:00,0.00,SYR,2020-01-01 20:45:00,SYR,19:30,19:20,80,71,-10,19,0,0,0,0,0,2020-01-01 19:30:00,2020-01-01 19:20:00,2020-01-01,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,2020-01-01 19:30:00,0.00,JFK,2020-01-01 19:30:00,early,early
4,B6,01/02/2020,116,JFK,08:22,08:06,75,67,-16,3,0,0,0,0,0,2020-01-02 08:22:00,2020-01-02 08:06:00,2020-01-02,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,2020-01-02 08:15:00,0.00,SYR,2020-01-02 08:15:00,SYR,07:07,06:59,75,67,-8,24,0,0,0,0,0,2020-01-02 07:07:00,2020-01-02 06:59:00,2020-01-02,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,2020-01-02 07:00:00,0.00,JFK,2020-01-02 07:00:00,early,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,UA,12/30/2023,783,ORD,23:59,00:13,114,104,14,5,0,0,0,0,0,2023-12-30 23:59:00,2023-12-31 00:13:00,2023-12-30,2023-12-30 23:59:00,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,2023-12-31 00:00:00,0.00,SYR,2023-12-31 00:00:00,SYR,21:05,21:29,114,104,24,17,0,0,0,0,0,2023-12-30 22:05:00,2023-12-30 22:29:00,2023-12-30,2023-12-30

In [4]:
df["scheduled_departure_time"] = pd.to_datetime(df["scheduled_departure_time"])
df["actual_departure_time"] = pd.to_datetime(df["actual_departure_time"])


df["scheduled_arrival_time"] = pd.to_datetime(df["scheduled_arrival_time"])
df["actual_arrival_time"] = pd.to_datetime(df["actual_arrival_time"])


In [5]:
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,timestamp_local_x,snow_rate_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,timestamp_local_y,snow_rate_y,key_y,date_time_map_y,arrival_status,previous_flight_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,2020-01-01 08:15:00,0.00,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,2020-01-01 07:00:00,0.00,JFK,2020-01-01 07:00:00,early,NaN
1,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,2020-01-01 11:30:00,0.00,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,2020-01-01 10:00:00,0.00,JFK,2020-01-01 10:00:00,early,early
2,YX,01/01/2020,5957,JFK,18:02,17:35,93,68,-27,4,0,0,0,0,0,2020-01-01 18:02:00,2020-01-01 17:35:00,2020-01-01,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,2020-01-01 18:00:00,0.00,SYR,2020-01-01 18:00:00,SYR,16:29,16:27,93,68,-2,13,0,0,0,0,0,2020-01-01 16:29:00,2020-01-01 16:27:00,2020-01-01,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,2020-01-01 16:30:00,0.00,JFK,2020-01-01 16:30:00,early,early
3,B6,01/01/2020,2516,JFK,20:50,20:31,80,71,-19,4,0,0,0,0,0,2020-01-01 20:50:00,2020-01-01 20:31:00,2020-01-01,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,2020-01-01 20:45:00,0.00,SYR,2020-01-01 20:45:00,SYR,19:30,19:20,80,71,-10,19,0,0,0,0,0,2020-01-01 19:30:00,2020-01-01 19:20:00,2020-01-01,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,2020-01-01 19:30:00,0.00,JFK,2020-01-01 19:30:00,early,early
4,B6,01/02/2020,116,JFK,08:22,08:06,75,67,-16,3,0,0,0,0,0,2020-01-02 08:22:00,2020-01-02 08:06:00,2020-01-02,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,2020-01-02 08:15:00,0.00,SYR,2020-01-02 08:15:00,SYR,07:07,06:59,75,67,-8,24,0,0,0,0,0,2020-01-02 07:07:00,2020-01-02 06:59:00,2020-01-02,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,2020-01-02 07:00:00,0.00,JFK,2020-01-02 07:00:00,early,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,UA,12/30/2023,783,ORD,23:59,00:13,114,104,14,5,0,0,0,0,0,2023-12-30 23:59:00,2023-12-31 00:13:00,2023-12-30,2023-12-30 23:59:00,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,2023-12-31 00:00:00,0.00,SYR,2023-12-31 00:00:00,SYR,21:05,21:29,114,104,24,17,0,0,0,0,0,2023-12-30 22:05:00,2023-12-30 22:29:00,2023-12-30,2023-12-30

In [6]:
df[df["Origin Airport"]=="JFK"].sort_values(by="actual_arrival_time", ascending=True)

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,timestamp_local_x,snow_rate_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,timestamp_local_y,snow_rate_y,key_y,date_time_map_y,arrival_status,previous_flight_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,2020-01-01 08:15:00,0.00,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,2020-01-01 07:00:00,0.00,JFK,2020-01-01 07:00:00,early,NaN
1,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,2020-01-01 11:30:00,0.00,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,2020-01-01 10:00:00,0.00,JFK,2020-01-01 10:00:00,early,early
2,YX,01/01/2020,5957,JFK,18:02,17:35,93,68,-27,4,0,0,0,0,0,2020-01-01 18:02:00,2020-01-01 17:35:00,2020-01-01,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,2020-01-01 18:00:00,0.00,SYR,2020-01-01 18:00:00,SYR,16:29,16:27,93,68,-2,13,0,0,0,0,0,2020-01-01 16:29:00,2020-01-01 16:27:00,2020-01-01,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,2020-01-01 16:30:00,0.00,JFK,2020-01-01 16:30:00,early,early
3,B6,01/01/2020,2516,JFK,20:50,20:31,80,71,-19,4,0,0,0,0,0,2020-01-01 20:50:00,2020-01-01 20:31:00,2020-01-01,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,2020-01-01 20:45:00,0.00,SYR,2020-01-01 20:45:00,SYR,19:30,19:20,80,71,-10,19,0,0,0,0,0,2020-01-01 19:30:00,2020-01-01 19:20:00,2020-01-01,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,2020-01-01 19:30:00,0.00,JFK,2020-01-01 19:30:00,early,early
4,B6,01/02/2020,116,JFK,08:22,08:06,75,67,-16,3,0,0,0,0,0,2020-01-02 08:22:00,2020-01-02 08:06:00,2020-01-02,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,2020-01-02 08:15:00,0.00,SYR,2020-01-02 08:15:00,SYR,07:07,06:59,75,67,-8,24,0,0,0,0,0,2020-01-02 07:07:00,2020-01-02 06:59:00,2020-01-02,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,2020-01-02 07:00:00,0.00,JFK,2020-01-02 07:00:00,early,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4568,9E,12/30/2023,4946,JFK,17:28,17:00,98,75,-28,4,0,0,0,0,0,2023-12-30 17:28:00,2023-12-30 17:00:00,2023-12-30,2023-12-30 17:28:00,-5.60,-1.70,0.00,100,-1.70,0.00,6,741,10.20,3.10,2023-12-30 17:30:00,0.00,SYR,2023-12-30 17:30:00,SYR,15:50,15:45,98,75,-5,15,0,0,0,0,0,2023-12-30 15:50:00,2023-12-30 15:45:00,2023-12-30,2023-12-30 1

In [7]:
df.columns

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', 'Origin Airport',
       'Scheduled Arrival Time', 'Actual Arrival Time',
       'Scheduled Elapsed Time (Minutes)', 'Actual Elapsed Time (Minutes)',
       'Arrival Delay (Minutes)', 'Taxi-In time (Minutes)',
       'Delay Carrier (Minutes)_x', 'Delay Weather (Minutes)_x',
       'Delay National Aviation System (Minutes)_x',
       'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
       'scheduled_arrival_time', 'actual_arrival_time', 'date_x',
       'date_time_x', 'app_temp_x', 'dewpt_x', 'precip_rate_x', 'rh_x',
       'temp_x', 'uv_x', 'vis_x', 'weather.code_x', 'wind_gust_spd_x',
       'wind_spd_x', 'timestamp_local_x', 'snow_rate_x', 'key_x',
       'date_time_map_x', 'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Taxi-Out time (Minutes)', 'Delay C

In [8]:
numeric_cols = df.select_dtypes(include='number').columns
numeric_cols

Index(['Flight Number', 'Scheduled Elapsed Time (Minutes)',
       'Actual Elapsed Time (Minutes)', 'Arrival Delay (Minutes)',
       'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x',
       'Delay Weather (Minutes)_x',
       'Delay National Aviation System (Minutes)_x',
       'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
       'app_temp_x', 'dewpt_x', 'precip_rate_x', 'rh_x', 'temp_x', 'uv_x',
       'vis_x', 'weather.code_x', 'wind_gust_spd_x', 'wind_spd_x',
       'snow_rate_x', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)_y',
       'Delay Weather (Minutes)_y',
       'Delay National Aviation System (Minutes)_y',
       'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y',
       'app_temp_y', 'dewpt_y', 'precip_rate_y', 'rh_y', 'temp_y', 'uv_y',
       'vis_y', 'weather.code_y', 'wind_gust_spd_y', 'wind_spd_y',

In [9]:
cat_cols = df.select_dtypes(exclude='number').columns
cat_cols

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Origin Airport',
       'Scheduled Arrival Time', 'Actual Arrival Time',
       'scheduled_arrival_time', 'actual_arrival_time', 'date_x',
       'date_time_x', 'timestamp_local_x', 'key_x', 'date_time_map_x',
       'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'scheduled_departure_time',
       'actual_departure_time', 'date_y', 'date_time_y', 'timestamp_local_y',
       'key_y', 'date_time_map_y', 'arrival_status', 'previous_flight_status'],
      dtype='object')

In [10]:
# df.drop(columns=["Date (MM/DD/YYYY)", "Scheduled Arrival Time", "Actual Elapsed Time (Minutes)", "Taxi-In time (Minutes)", "Delay Carrier (Minutes)_x", "Delay Weather (Minutes)_x", "Delay National Aviation System (Minutes)_x", "Delay Security (Minutes)_x", "Delay Late Aircraft Arrival (Minutes)_x", "actual_arrival_time", "date_x", "date_time_x", "date_time_map_x", "key_x", "Destination Airport", "Scheduled departure time", ""])


cat_drop_cols = ["Date (MM/DD/YYYY)", "Scheduled Arrival Time", "Actual Arrival Time", "date_x", 
             "date_time_x", "timestamp_local_x", "key_x", "date_time_map_x", "Destination Airport", 
             "Scheduled departure time", "Actual departure time",'date_y', 'date_time_y', 'timestamp_local_y',
             'key_y', 'date_time_map_y' 
            ]

numeric_drop_cols = ['Flight Number', 'Actual Elapsed Time (Minutes)',
                     'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x','Delay Weather (Minutes)_x', 
                     'Delay National Aviation System (Minutes)_x', 'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
                     'Scheduled elapsed time (Minutes)','Actual elapsed time (Minutes)', 'Departure delay (Minutes)','Taxi-Out time (Minutes)', 
                     'Delay Carrier (Minutes)_y','Delay Weather (Minutes)_y','Delay National Aviation System (Minutes)_y',
                     'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y']

In [11]:
df['arrival_status'] = pd.cut(df['Arrival Delay (Minutes)'], bins=[float('-inf'), -5, 5.00001, float('inf')], labels=["early", "ontime", "late"], right=False)
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,timestamp_local_x,snow_rate_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,timestamp_local_y,snow_rate_y,key_y,date_time_map_y,arrival_status,previous_flight_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,2020-01-01 08:15:00,0.00,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,2020-01-01 07:00:00,0.00,JFK,2020-01-01 07:00:00,early,NaN
1,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,2020-01-01 11:30:00,0.00,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,2020-01-01 10:00:00,0.00,JFK,2020-01-01 10:00:00,early,early
2,YX,01/01/2020,5957,JFK,18:02,17:35,93,68,-27,4,0,0,0,0,0,2020-01-01 18:02:00,2020-01-01 17:35:00,2020-01-01,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,2020-01-01 18:00:00,0.00,SYR,2020-01-01 18:00:00,SYR,16:29,16:27,93,68,-2,13,0,0,0,0,0,2020-01-01 16:29:00,2020-01-01 16:27:00,2020-01-01,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,2020-01-01 16:30:00,0.00,JFK,2020-01-01 16:30:00,early,early
3,B6,01/01/2020,2516,JFK,20:50,20:31,80,71,-19,4,0,0,0,0,0,2020-01-01 20:50:00,2020-01-01 20:31:00,2020-01-01,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,2020-01-01 20:45:00,0.00,SYR,2020-01-01 20:45:00,SYR,19:30,19:20,80,71,-10,19,0,0,0,0,0,2020-01-01 19:30:00,2020-01-01 19:20:00,2020-01-01,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,2020-01-01 19:30:00,0.00,JFK,2020-01-01 19:30:00,early,early
4,B6,01/02/2020,116,JFK,08:22,08:06,75,67,-16,3,0,0,0,0,0,2020-01-02 08:22:00,2020-01-02 08:06:00,2020-01-02,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,2020-01-02 08:15:00,0.00,SYR,2020-01-02 08:15:00,SYR,07:07,06:59,75,67,-8,24,0,0,0,0,0,2020-01-02 07:07:00,2020-01-02 06:59:00,2020-01-02,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,2020-01-02 07:00:00,0.00,JFK,2020-01-02 07:00:00,early,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,UA,12/30/2023,783,ORD,23:59,00:13,114,104,14,5,0,0,0,0,0,2023-12-30 23:59:00,2023-12-31 00:13:00,2023-12-30,2023-12-30 23:59:00,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,2023-12-31 00:00:00,0.00,SYR,2023-12-31 00:00:00,SYR,21:05,21:29,114,104,24,17,0,0,0,0,0,2023-12-30 22:05:00,2023-12-30 22:29:00,2023-12-30,2023-12-30

In [12]:
model_df = df.drop(columns=cat_drop_cols + numeric_drop_cols + ["Arrival Delay (Minutes)", "actual_arrival_time", "actual_departure_time"]).copy()
# model_df = df.drop(columns=cat_drop_cols+numeric_drop_cols).copy()
# model_df.to_csv("./data/modified/test.csv", index=False)
# carrier_code_map = {
#     "JFK": {
#         "9E": "DL"
#     },
#     "MCO": {
        
#     },
#     "ORD": {
        
#     }
# }

# model_df.loc[(model_df["Origin Airport"]=="JFK")&(), "Carrier Code"] = ""
# model_df.loc[(model_df["Origin Airport"]=="MCO")&(), "Carrier Code"] = ""
# model_df.loc[(model_df["Origin Airport"]=="ORD")&(), "Carrier Code"] = ""


In [13]:
numeric_mod_cols = model_df.select_dtypes(include="number").columns
cat_mod_cols = model_df.select_dtypes(exclude="number").columns

In [14]:
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),scheduled_arrival_time,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,snow_rate_x,scheduled_departure_time,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,snow_rate_y,arrival_status,previous_flight_status
0,B6,JFK,75,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,0.00,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,0.00,early,NaN
1,9E,JFK,92,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,0.00,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,0.00,early,early
2,YX,JFK,93,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,0.00,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,0.00,early,early
3,B6,JFK,80,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,0.00,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,0.00,early,early
4,B6,JFK,75,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,0.00,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,0.00,early,early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,UA,ORD,114,2023-12-30 23:59:00,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,0.00,2023-12-30 22:05:00,-0.90,0.60,0.00,89,2.20,0.00,14,804,5.60,3.10,0.00,late,ontime
11776,MQ,ORD,114,2023-12-31 11:19:00,-3.00,-2.20,0.00,92,-1.10,0.80,11,804,5.40,1.50,0.00,2023-12-31 09:25:00,-4.30,-1.20,1.00,96,-0.60,0.40,2,600,8.60,3.10,17.00,late,late
11777,YX,ORD,124,2023-12-31 16:44:00,-5.30,-3.40,0.00,88,-1.70,0.00,14,804,2.72,2.72,0.00,2023-12-31 14:40:00,-2.40,-2.10,0.00,80,1.00,0.50,4,804,7.40,3.10,0.00,late,late
11778,MQ,ORD,106,2023-12-31 17:28:00,-4.50,-3.70,0.00,86,-1.70,0.00,15,804,2.04,2.04,0.00,2023-12-31 15:42:00,-2.30,-2.30,0.00,78,1.10,0.40,10,804,6.60,3.10,0.00,late,late


In [15]:
model_df.columns

Index(['Carrier Code', 'Origin Airport', 'Scheduled Elapsed Time (Minutes)',
       'scheduled_arrival_time', 'app_temp_x', 'dewpt_x', 'precip_rate_x',
       'rh_x', 'temp_x', 'uv_x', 'vis_x', 'weather.code_x', 'wind_gust_spd_x',
       'wind_spd_x', 'snow_rate_x', 'scheduled_departure_time', 'app_temp_y',
       'dewpt_y', 'precip_rate_y', 'rh_y', 'temp_y', 'uv_y', 'vis_y',
       'weather.code_y', 'wind_gust_spd_y', 'wind_spd_y', 'snow_rate_y',
       'arrival_status', 'previous_flight_status'],
      dtype='object')

In [16]:
model_df.dtypes

Carrier Code                                object
Origin Airport                              object
Scheduled Elapsed Time (Minutes)             int64
scheduled_arrival_time              datetime64[ns]
app_temp_x                                 float64
dewpt_x                                    float64
precip_rate_x                              float64
rh_x                                         int64
temp_x                                     float64
uv_x                                       float64
vis_x                                        int64
weather.code_x                               int64
wind_gust_spd_x                            float64
wind_spd_x                                 float64
snow_rate_x                                float64
scheduled_departure_time            datetime64[ns]
app_temp_y                                 float64
dewpt_y                                    float64
precip_rate_y                              float64
rh_y                           

In [17]:
model_df['scheduled_departure_hour'] = model_df['scheduled_departure_time'].dt.hour
model_df['scheduled_departure_day'] = model_df['scheduled_departure_time'].dt.day
model_df['scheduled_departure_month'] = model_df['scheduled_departure_time'].dt.month
model_df['scheduled_departure_weekday'] = model_df['scheduled_departure_time'].dt.weekday

model_df['scheduled_arrival_hour'] = model_df['scheduled_arrival_time'].dt.hour
model_df['scheduled_arrival_day'] = model_df['scheduled_arrival_time'].dt.day
model_df['scheduled_arrival_month'] = model_df['scheduled_arrival_time'].dt.month
model_df['scheduled_arrival_weekday'] = model_df['scheduled_arrival_time'].dt.weekday

weather_features = ['app_temp', 'dewpt', 'precip_rate', 'rh', 'temp', 'uv', 'vis', 'weather.code', 'wind_gust_spd', 'wind_spd', 'snow_rate']
for feature in weather_features:
    model_df[f'{feature}_diff'] = model_df[f'{feature}_x'] - model_df[f'{feature}_y']


carrier_dummies = pd.get_dummies(model_df['Carrier Code'], prefix='carrier')
airport_dummies = pd.get_dummies(model_df['Origin Airport'], prefix='airport')


model_df = pd.concat([model_df, carrier_dummies, airport_dummies], axis=1)



In [18]:
import numpy as np

model_df['dep_time_day_prop'] = model_df['scheduled_departure_time'].dt.hour * 60 + model_df['scheduled_departure_time'].dt.minute
model_df['dep_time_day_prop'] /= 1440  

model_df['arr_time_day_prop'] = model_df['scheduled_arrival_time'].dt.hour * 60 + model_df['scheduled_arrival_time'].dt.minute
model_df['arr_time_day_prop'] /= 1440  


model_df['night_flight'] = model_df['scheduled_departure_time'].dt.hour.apply(lambda x: 1 if x >= 20 or x < 6 else 0)

model_df['extreme_weather_temp_low'] = model_df['temp_x'].apply(lambda x: 1 if x < -10 else 0)
model_df['extreme_weather_temp_high'] = model_df['temp_x'].apply(lambda x: 1 if x > 35 else 0)
model_df['extreme_weather_wind'] = model_df['wind_gust_spd_x'].apply(lambda x: 1 if x > 50 else 0)
model_df['extreme_weather_snow'] = model_df['snow_rate_x'].apply(lambda x: 1 if x > 0.5 else 0)  # More than 0.5 mm/h

model_df[['dep_time_day_prop', 'arr_time_day_prop', 'night_flight', 
      'extreme_weather_temp_low', 'extreme_weather_temp_high', 'extreme_weather_wind', 'extreme_weather_snow']].head()


,dep_time_day_prop,arr_time_day_prop,night_flight,extreme_weather_temp_low,extreme_weather_temp_high,extreme_weather_wind,extreme_weather_snow
0,0.30,0.35,0,0,0,0,0
1,0.42,0.48,0,0,0,0,0
2,0.69,0.75,0,0,0,0,0
3,0.81,0.87,0,0,0,0,0
4,0.30,0.35,0,0,0,0,0


In [19]:
def categorize_hour(hour):
    if 0 <= hour <= 3:
        return 'Early Morning'
    elif 4 <= hour <= 7:
        return 'Morning'
    elif 8 <= hour <= 11:
        return 'Late Morning'
    elif 12 <= hour <= 15:
        return 'Afternoon'
    elif 16 <= hour <= 19:
        return 'Evening'
    elif 20 <= hour <= 23:
        return 'Night'

model_df['time_of_day'] = model_df['scheduled_arrival_hour'].apply(categorize_hour)

In [20]:
model_df["peak_zone"] = "peak"

In [21]:
model_df.loc[((model_df['time_of_day']=="Morning") | (model_df['time_of_day']=="Early Morning") | (model_df['time_of_day']=="Afternoon")), "peak_zone"] = "off_peak"

In [22]:
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),scheduled_arrival_time,app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,snow_rate_x,scheduled_departure_time,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,snow_rate_y,arrival_status,previous_flight_status,scheduled_departure_hour,scheduled_departure_day,scheduled_departure_month,scheduled_departure_weekday,scheduled_arrival_hour,scheduled_arrival_day,scheduled_arrival_month,scheduled_arrival_weekday,app_temp_diff,dewpt_diff,precip_rate_diff,rh_diff,temp_diff,uv_diff,vis_diff,weather.code_diff,wind_gust_spd_diff,wind_spd_diff,snow_rate_diff,carrier_9E,carrier_AA,carrier_B6,carrier_F9,carrier_MQ,carrier_OO,carrier_UA,carrier_WN,carrier_YX,airport_JFK,airport_MCO,airport_ORD,dep_time_day_prop,arr_time_day_prop,night_flight,extreme_weather_temp_low,extreme_weather_temp_high,extreme_weather_wind,extreme_weather_snow,time_of_day,peak_zone
0,B6,JFK,75,2020-01-01 08:22:00,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,0.00,2020-01-01 07:07:00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,0.00,early,NaN,7,1,1,2,8,1,1,2,-3.90,-3.90,0.00,-4,-3.20,0.40,0,0,1.00,-0.63,0.00,False,False,True,False,False,False,False,False,False,True,False,False,0.30,0.35,0,0,0,0,0,Late Morning,peak
1,9E,JFK,92,2020-01-01 11:37:00,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,0.00,2020-01-01 10:05:00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,0.00,early,early,10,1,1,2,11,1,1,2,-3.90,0.40,0.00,14,-3.10,-0.10,0,0,1.00,0.05,0.00,True,False,False,False,False,False,False,False,False,True,False,False,0.42,0.48,0,0,0,0,0,Late Morning,peak
2,YX,JFK,93,2020-01-01 18:02:00,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,0.00,2020-01-01 16:29:00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,0.00,early,early,16,1,1,2,18,1,1,2,-3.70,0.10,0.00,15,-3.50,-0.50,0,0,-3.20,-1.99,0.00,False,False,False,False,False,False,False,False,True,True,False,False,0.69,0.75,0,0,0,0,0,Evening,peak
3,B6,JFK,80,2020-01-01 20:50:00,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,0.00,2020-01-01 19:30:00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,0.00,early,early,19,1,1,2,20,1,1,2,-1.40,0.80,0.00,14,-2.40,0.00,0,1,-3.10,-3.31,0.00,False,False,True,False,False,False,False,False,False,True,False,False,0.81,0.87,0,0,0,0,0,Night,peak
4,B6,JFK,75,2020-01-02 08:22:00,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,0.00,2020-01-02 07:07:00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,0.00,early,early,7,2,1,3,8,2,1,3,4.30,-0.70,0.00,-18,2.90,0.50,0,2,1.30,-0.99,0.00,False,False,True,False,False,False,False,False,False,True,False,False,0.30,0.35,0,0,0,0,0,Late Morning,peak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,UA,ORD,114,2023-12-30 23:59:00,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,0.00,2023-12-30 22:05:00,-0.90,0.60,0.00,89,2.20,0.00,14,804,5.60,3.10,0.00,late,ontime,22,30,12,5,23,30,12,5,-5.90,-3.40,0.00,3,-3.90,0.00,0,0,1.60,1.49,0.00,False,False,False,False,False,False,True,False,False,False,False,True,0.92,1.00,1,0,0,0,0,Night,peak
11776,MQ,ORD,114,2023-12-31 11:19:00,-3.00,-2.20,0.00,92,-1.10,0.80,11,804,5.40,1.50,0.00,2023-12-31 09:25:00,-4.30,-1.20,1.00,96,-0.60,0.40,2,600,8.60,3.10,17.00,late,late,9,31,12,6,11,31,12,6,1.30,-1.00,-1.00,-4,-0.50,0.40,9,204,-3.20,-1.60,-17.00,False,False,False,False,True,False,False,False,False,False,False,True,0.39,0.47,0,0,0,0,0,Late Morning,peak
11777,YX,ORD,124,2023-12-31 16:44:00,-5.30,-3.40,0.00,88,-1.70,0.00,14,804,2.72,2.72,0.00,2023-12-31 14:40:00,-2.40,-2.10,0.00,80,1.00,0.50,4,804,7.40,3.10,0.00,late,late,14,31,12,6,16,31,12,6,-2.90,-1.30,0.00,8,-2.70,-0.50,10,0,-4.68,-0.38,0.00,False,False,False,False,False,False,False,False,Tr

In [23]:
categorical_cols = ['time_of_day', 'peak_zone', 'previous_flight_status']
model_df = pd.get_dummies(model_df, columns=categorical_cols)

In [24]:
bool_cols = model_df.select_dtypes(include=["bool"]).columns
for i in bool_cols:
    model_df[i] *= 1
model_df[bool_cols]

,carrier_9E,carrier_AA,carrier_B6,carrier_F9,carrier_MQ,carrier_OO,carrier_UA,carrier_WN,carrier_YX,airport_JFK,airport_MCO,airport_ORD,time_of_day_Afternoon,time_of_day_Early Morning,time_of_day_Evening,time_of_day_Late Morning,time_of_day_Morning,time_of_day_Night,peak_zone_off_peak,peak_zone_peak,previous_flight_status_early,previous_flight_status_late,previous_flight_status_ontime
0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0
3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0
4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1
11776,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0
11777,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0
11778,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0


In [25]:
bool_cols

Index(['carrier_9E', 'carrier_AA', 'carrier_B6', 'carrier_F9', 'carrier_MQ',
       'carrier_OO', 'carrier_UA', 'carrier_WN', 'carrier_YX', 'airport_JFK',
       'airport_MCO', 'airport_ORD', 'time_of_day_Afternoon',
       'time_of_day_Early Morning', 'time_of_day_Evening',
       'time_of_day_Late Morning', 'time_of_day_Morning', 'time_of_day_Night',
       'peak_zone_off_peak', 'peak_zone_peak', 'previous_flight_status_early',
       'previous_flight_status_late', 'previous_flight_status_ontime'],
      dtype='object')

In [26]:
drop_cols = model_df.select_dtypes(exclude=["number"]).columns
drop_cols

Index(['Carrier Code', 'Origin Airport', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_status'],
      dtype='object')

In [27]:
selected_features = model_df.select_dtypes(include=["number"]).columns
len(selected_features)

72

In [28]:
# # Define the target variable based on arrival delay
# df['Class of Arrival Time'] = pd.cut(df['Arrival Delay (Minutes)_a'],
#                                        bins=[float('-inf'), -5, 5, float('inf')],
#                                        labels=['Early', 'On-time', 'Late'])

# Drop unnecessary columns and the target variable
X = model_df.drop(columns=drop_cols, axis=1)
y = model_df['arrival_status']
# y = df['Arrival Delay (Minutes)_a']


# Convert categorical variables to numerical using one-hot encoding
# categorical_cols = ['Origin Airport_a', 'wx_phrase_a', 'uv_desc_a', 'clds_a',
#                     'wx_phrase_d', 'uv_desc_d', 'clds_d', 'Main_Carrier', 'Week_Category', 'A_Time_of_Day']
# X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# scaler = StandardScaler()
# X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
# X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [29]:
X_encoded = X

# Standardize numerical columns
numerical_cols = selected_features
scaler = StandardScaler()


In [30]:
X_encoded[selected_features]

,Scheduled Elapsed Time (Minutes),app_temp_x,dewpt_x,precip_rate_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_gust_spd_x,wind_spd_x,snow_rate_x,app_temp_y,dewpt_y,precip_rate_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_gust_spd_y,wind_spd_y,snow_rate_y,scheduled_departure_hour,scheduled_departure_day,scheduled_departure_month,scheduled_departure_weekday,scheduled_arrival_hour,scheduled_arrival_day,scheduled_arrival_month,scheduled_arrival_weekday,app_temp_diff,dewpt_diff,precip_rate_diff,rh_diff,temp_diff,uv_diff,vis_diff,weather.code_diff,wind_gust_spd_diff,wind_spd_diff,snow_rate_diff,carrier_9E,carrier_AA,carrier_B6,carrier_F9,carrier_MQ,carrier_OO,carrier_UA,carrier_WN,carrier_YX,airport_JFK,airport_MCO,airport_ORD,dep_time_day_prop,arr_time_day_prop,night_flight,extreme_weather_temp_low,extreme_weather_temp_high,extreme_weather_wind,extreme_weather_snow,time_of_day_Afternoon,time_of_day_Early Morning,time_of_day_Evening,time_of_day_Late Morning,time_of_day_Morning,time_of_day_Night,peak_zone_off_peak,peak_zone_peak,previous_flight_status_early,previous_flight_status_late,previous_flight_status_ontime
0,75,-6.50,-6.20,0.00,65,-0.40,0.40,16,804,12.10,7.07,0.00,-2.60,-2.30,0.00,69,2.80,0.00,16,804,11.10,7.70,0.00,7,1,1,2,8,1,1,2,-3.90,-3.90,0.00,-4,-3.20,0.40,0,0,1.00,-0.63,0.00,0,0,1,0,0,0,0,0,0,1,0,0,0.30,0.35,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1,92,-5.10,-5.40,0.00,63,0.80,0.60,16,804,12.60,7.75,0.00,-1.20,-5.80,0.00,49,3.90,0.70,16,804,11.60,7.70,0.00,10,1,1,2,11,1,1,2,-3.90,0.40,0.00,14,-3.10,-0.10,0,0,1.00,0.05,0.00,1,0,0,0,0,0,0,0,0,1,0,0,0.42,0.48,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
2,93,-5.50,-5.80,0.00,65,0.00,0.00,16,804,8.40,6.20,0.00,-1.80,-5.90,0.00,50,3.50,0.50,16,804,11.60,8.19,0.00,16,1,1,2,18,1,1,2,-3.70,0.10,0.00,15,-3.50,-0.50,0,0,-3.20,-1.99,0.00,0,0,0,0,0,0,0,0,1,1,0,0,0.69,0.75,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0
3,80,-3.90,-5.00,0.00,67,0.40,0.00,16,804,8.10,4.22,0.00,-2.50,-5.80,0.00,53,2.80,0.00,16,803,11.20,7.53,0.00,19,1,1,2,20,1,1,2,-1.40,0.80,0.00,14,-2.40,0.00,0,1,-3.10,-3.31,0.00,0,0,1,0,0,0,0,0,0,1,0,0,0.81,0.87,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0
4,75,-0.40,-4.10,0.00,60,2.90,0.50,16,804,8.10,3.60,0.00,-4.70,-3.40,0.00,78,0.00,0.00,16,802,6.80,4.59,0.00,7,2,1,3,8,2,1,3,4.30,-0.70,0.00,-18,2.90,0.50,0,2,1.30,-0.99,0.00,0,0,1,0,0,0,0,0,0,1,0,0,0.30,0.35,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,114,-6.80,-2.80,0.00,92,-1.70,0.00,14,804,7.20,4.59,0.00,-0.90,0.60,0.00,89,2.20,0.00,14,804,5.60,3.10,0.00,22,30,12,5,23,30,12,5,-5.90,-3.40,0.00,3,-3.90,0.00,0,0,1.60,1.49,0.00,0,0,0,0,0,0,1,0,0,0,0,1,0.92,1.00,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
11776,114,-3.00,-2.20,0.00,92,-1.10,0.80,11,804,5.40,1.50,0.00,-4.30,-1.20,1.00,96,-0.60,0.40,2,600,8.60,3.10,17.00,9,31,12,6,11,31,12,6,1.30,-1.00,-1.00,-4,-0.50,0.40,9,204,-3.20,-1.60,-17.00,0,0,0,0,1,0,0,0,0,0,0,1,0.39,0.47,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
11777,124,-5.30,-3.40,0.00,88,-1.70,0.00,14,804,2.72,2.72,0.00,-2.40,-2.10,0.00,80,1.00,0.50,4,804,7.40,3.10,0.00,14,31,12,6,16,31,12,6,-2.90,-1.30,0.00,8,-2.70,-0.50,10,0,-4.68,-0.38,0.00,0,0,0,0,0,0,0,0,1,0,0,1,0.61,0.70,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
11778,106,-4.50,-3.70,0.00,86,-1.70,0.00,15,804,2.04,2.04,0.00,-2.30,-2.30,0.00,78,1.10,0.40,10,804,6.60,3.10,0.00,15,31,12,6,17,31,12,6,-2.20,-1.40,0.00,8,-2.80,-0.40,5,0,-4.56,-1.06,0.00,0,0,0,0,1,0,0,0,0,0,0,1,0.65,0.73,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0


In [31]:
X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score



# Initialize and train different models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42,n_estimators = 250,max_depth = 50, learning_rate= 1 ),
    'ada' : AdaBoostClassifier(random_state=42),
    # 'xg':  XGBClassifier(random_state=42),
    'knn' : KNeighborsClassifier(n_neighbors=15),
    'dtree':  DecisionTreeClassifier(max_depth=7,random_state=42),
    'rand_class':  RandomForestClassifier(max_depth=15,random_state=42),
    'GNB':  GaussianNB()
}

for model_name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'{model_name} Accuracy: {accuracy:.2f}')
        # mae = mean_absolute_error(y_test, y_pred)
        # mae_ratio = mae / np.mean(y_test)
        # print("MAE:", mae)
        # print("MAE Ratio:", mae_ratio)
        # print(model.score(X_test,y_test))
        # print(classification_report(y_test, y_pred))
        # print('-' * 50)
    except Exception as e:
        print(e)

/Users/saikeerthanpalavarapu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Logistic Regression Accuracy: 0.57


DecisionTreeClassifier(random_state=42)

Decision Tree Accuracy: 0.45


...

dtree Accuracy: 0.53


RandomForestClassifier(max_depth=15, random_state=42)

rand_class Accuracy: 0.57


GaussianNB()

GNB Accuracy: 0.29
